In [1]:
%load_ext autoreload
%autoreload 2
    
%load_ext tensorboard
import sys, os
sys.path.append('../paraphrase/')
from paraphraser_args import ModelArguments, DataTrainingArguments, TrainingArguments
from paraphraser_dataloader import load_dataset
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead, HfArgumentParser

In [2]:
data_dir = "../data/paraphrase/paranmt_filtered"
model_name = "t5-small"
model_nick = "t5_paraphrase"
output_dir = "../models/"
epochs = "12"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "400"

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True",
    "--meta_task",
    "paraphrase"
])


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelWithLMHead.from_pretrained(model_args.model_name_or_path)

loading configuration file https://huggingface.co/Vamsi/T5_Paraphrase_Paws/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/efb069f5209561e0d66ba08840c953b476b2b25058b7648dc689234c60af2b24.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3

In [4]:
train_dataset = load_dataset(data_args.data_dir, tokenizer, mode="train", n_proc=2048)
dev_dataset = load_dataset(data_args.data_dir, tokenizer, mode="dev", n_proc=2048)

100%|██████████| 6/6 [00:01<00:00,  3.34it/s]


In [5]:
# Open TensorBoard
# %tensorboard --logdir runs

In [6]:
trainer = ParaphraserTrainer([training_args,model_args, data_args], model, tokenizer, train_dataset, dev_dataset)
trainer.train()

***** Running training *****
Num examples = 62469
Num Epochs = 10.0
Total train batch size = 16
Gradient Accumulation steps = 1
Total optimization steps = 39050.0
Logging steps = 400
Save steps = 400
Epoch:   0%|          | 0/10 [00:03<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 502.00 MiB (GPU 0; 23.65 GiB total capacity; 21.99 GiB already allocated; 431.06 MiB free; 22.16 GiB reserved in total by PyTorch)

In [57]:
txt = "Could you please open the door?"
t = tokenizer(txt, return_tensors='pt')
d = model.generate(input_ids=t.input_ids.cuda(), attention_mask=t.attention_mask.cuda(), temperature=2)

In [58]:
tokenizer.decode(d[0], skip_special_tokens=True)

'please open the door?'